In [1]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import cv2
from os import listdir
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

def unique(list1): 
    # insert the list to the set 
    list_set = set(list1) 
    # convert the set to the list 
    unique_list = (list(list_set)) 
    return unique_list

import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
from statsmodels.stats import multitest
from statsmodels.sandbox.stats.multicomp import MultiComparison

In [2]:
folder = '/Users/margaretschroeder/Dropbox (MIT)/BoydenLab/ExR_Decrowding/Decrowding crop image/'

In [3]:
data_raw = pd.read_csv(folder + 'synapselevel_decrowding_forstats_20210306.csv')

In [4]:
data_raw

,protein,pre_post,in_out,signal,volume
0,Cav2.1,pre,out,4.674356,635.500000
1,Cav2.1,pre,out,2.382948,9.872727
2,Cav2.1,pre,out,3.599700,29.960000
3,Cav2.1,pre,out,62.292006,249.411765
4,Cav2.1,pre,out,60.592382,130.517857
...,...,...,...,...,...
244,Shank3,post,in,98.754629,1701.181818
245,Shank3,post,in,83.498446,1928.047619
246,Shank3,post,in,95.127475,2145.064516
247,Shank3,post,in,95.350465,2900.079365


## Run three-way ANOVA with protein, in/out, and pre/post as factors, and no interaction terms

In [26]:
#change the model between signal and volume below
lm_full = ols('volume ~ C(protein) + C(in_out) + C(pre_post)',data=data_raw).fit()
table_full = sm.stats.anova_lm(lm_full, typ=3)
print(table_full)
table_full.to_clipboard()

                   sum_sq     df           F        PR(>F)
Intercept    5.820695e+07    1.0  177.796700  1.077695e-30
C(protein)   2.048699e+07    6.0   10.429795  2.859230e-10
C(in_out)    2.357308e+07    1.0   72.005420  2.274974e-15
C(pre_post)  1.540138e+07    1.0   47.044455  5.892117e-11
Residual     7.824364e+07  239.0         NaN           NaN


## Now run multiple 2-way ANOVAs for each protein, and correct these

In [8]:
prot_list = ['Bassoon','Cav2.1','Homer1','PSD95','Rim1/2','Shank3','SynGAP']

In [39]:
#volume
pvals = []
for prot in prot_list:
    prot_subset = data_raw[data_raw['protein']==prot]
    lm = ols('volume ~ C(pre_post) + C(in_out)',data=prot_subset).fit()
    tbl = sm.stats.anova_lm(lm, typ=3)
    p = tbl.loc['C(in_out)']['PR(>F)']
    pvals.append(p)
    print(prot)
    print(p)

Bassoon
0.040855913068770865
Cav2.1
0.319983380550952
Homer1
1.710335080052583e-11
PSD95
0.032895349735568805
Rim1/2
0.001399629274468057
Shank3
9.649889733595324e-10
SynGAP
0.023985849075715472


In [40]:
pvalarr = np.array(pvals)

In [41]:
reject,pvalscorr,_,_ = multitest.multipletests(pvalarr, alpha=0.05, method='hs', is_sorted=False, returnsorted=False)

In [42]:
vol_corrected_multiways = pd.DataFrame([pvalarr,pvalscorr,reject])

In [43]:
vol_corrected_multiways.T.to_clipboard()

In [44]:
vol_corrected_multiways.T

,0,1,2
0,0.0408559,0.0954753,False
1,0.319983,0.319983,False
2,1.71034e-11,1.19723e-10,True
3,0.0328953,0.0954753,False
4,0.00139963,0.00697858,True
5,9.64989e-10,5.78993e-09,True
6,0.0239858,0.0925463,False
